In [71]:
from pathlib import Path
from scipy.io import mmread
from scipy.sparse import coo_matrix
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader

WORKSPACE_ROOT = Path("../../").resolve()
Path("./cache").mkdir(exist_ok=True)

In [10]:
adata = ad.read_h5ad("../scanpy/cache/adata.h5ad")
adata

AnnData object with n_obs × n_vars = 25312 × 36601
    obs: 'barcode', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'leiden', 'cell_type', 'sample_type'
    var: 'locus', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'sample_colors', 'sample_type_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'ora_estimate', 'ora_pvals'
    varm: 'PCs'
    layers: 'log_norm'
    obsp: 'connectivities', 'distances'

In [87]:
go_dag = GODag(WORKSPACE_ROOT/"data/goatools/go-basic.obo")

/home/tony/workspace/rt/ctap/data/goatools/go-basic.obo: fmt(1.2) rel(2024-10-27) 44,017 Terms


In [97]:
columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID', 'DB_Reference',
    'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name', 'DB_Object_Synonym',
    'DB_Object_Type', 'Taxon_ID', 'Date', 'Assigned_By', 'Annotation_Extension', 'Gene_Product_Form_ID'
]
dfg2g = pd.read_csv(WORKSPACE_ROOT/"data/goatools/goa_human.gaf", sep="\t", comment="!", header=None)
dfg2g.columns = columns
print(dfg2g.shape)
dfg2g.head(2)

(782823, 17)


,DB,DB_Object_ID,DB_Object_Symbol,Qualifier,GO_ID,DB_Reference,Evidence_Code,With_From,Aspect,DB_Object_Name,DB_Object_Synonym,DB_Object_Type,Taxon_ID,Date,Assigned_By,Annotation_Extension,Gene_Product_Form_ID
0,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0003723,GO_REF:0000043,IEA,UniProtKB-KW:KW-0694,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20241014,UniProt,NaN,UniProtKB:A0A024RBG1
1,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0005515,PMID:33961781,IPI,UniProtKB:Q8NFP7,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20241012,IntAct,NaN,UniProtKB:A0A024RBG1


In [155]:
mapped_genes = set(dfg2g.DB_Object_Symbol)
print(len(mapped_genes))
go_filter = np.array([v in mapped_genes for v in adata.var.index])
print(go_filter.sum())

44541
18676


In [159]:
def get_parents_at(go_id, level=3):
    e = go_dag[go_id]
    def _traverse(e):
        if e.level == level:
            return {e}
        if e.level < level:
            return set()
        parents = set()
        for p in e.parents:
            parents |= _traverse(p)
        return parents
    return _traverse(e)

get_parents_at("GO:0048666")

{GOTerm('GO:0048468'):
   id:GO:0048468
   item_id:GO:0048468
   name:cell development
   namespace:biological_process
   _parents: 2 items
     GO:0048869
     GO:0048856
   parents: 2 items
     GO:0048869	level-02	depth-02	cellular developmental process [biological_process]
     GO:0048856	level-02	depth-02	anatomical structure development [biological_process]
   children: 42 items
   level:3
   depth:3
   is_obsolete:False
   alt_ids: 0 items}

In [165]:
k = 5
for cell_type in adata.obs["cell_type"].unique():
    _ss = adata[adata.obs["cell_type"] == cell_type]
    _ss = _ss[:, go_filter]

    # shortlist top k genes
    inf = _ss[_ss.obs["sample_type"] == "Infected"]
    ctl = _ss[_ss.obs["sample_type"] == "Control"]
    fc = np.array(inf.X.tocsc().sum(axis=0))[0] / np.array(ctl.X.tocsc().sum(axis=0))[0]
    topKi = np.argpartition(fc, -k)[-k:]
    topK = _ss.var.iloc[topKi].index.tolist()

    # get go terms for top k genes and aggregate at level 3
    gos = dfg2g[dfg2g["DB_Object_Symbol"].isin(topK)]["GO_ID"].unique()
    gos = {x.name for g in gos for x in get_parents_at(g, 3)}
    break
gos

{'G protein-coupled receptor signaling pathway',
 'active transmembrane transporter activity',
 'catabolic process',
 'defense response to other organism',
 'detection of chemical stimulus',
 'detection of stimulus involved in sensory perception',
 'extracellular organelle',
 'innate immune response',
 'inorganic ion transmembrane transport',
 'inorganic molecular entity transmembrane transporter activity',
 'intracellular organelle',
 'ion binding',
 'ketone metabolic process',
 'macromolecule metabolic process',
 'membrane-bounded organelle',
 'monoatomic ion transmembrane transport',
 'monoatomic ion transmembrane transporter activity',
 'monooxygenase activity',
 'negative regulation of biological process',
 'nervous system process',
 'olefinic compound metabolic process',
 'organelle membrane',
 'oxidoreductase activity, acting on paired donors, with incorporation or reduction of molecular oxygen',
 'phosphorus metabolic process',
 'plasma membrane',
 'plasma membrane region',
 'p

In [127]:
# https://nbisweden.github.io/workshop-scRNAseq/labs/scanpy/scanpy_05_dge.html
# gsea!

['AF212831.1',
 'LINC01737',
 'AL121999.1',
 'LINC01166',
 'AL050403.1',
 'LINC00374',
 'GPC5-AS2',
 'LINC01677',
 'AP001062.2',
 'AL591719.2',
 'CYP1B1-AS1',
 'AL355537.1',
 'AL139081.1',
 'GACAT1',
 'AL445183.1',
 'AC005324.1',
 'AC018742.1',
 'NALCN-AS1',
 'AL121832.1',
 'AL669841.1',
 'AC244453.2',
 'AC011899.2',
 'PHGR1',
 'AL049820.1',
 'AL138899.3']

In [55]:
topK

['RPS12',
 'EEF1A1',
 'RPL10',
 'PTMA',
 'MALAT1',
 'MT-ATP6',
 'B2M',
 'MT-CO2',
 'TMSB10',
 'RPS27']

In [46]:
_ss.X.tocsc().sum(axis=0)

matrix([[0.       , 0.       , 1.2376149, ..., 0.       , 0.       ,
         0.       ]])

In [6]:
gp.profile(organism='hsapiens',
    query=['NR1H4','TRIP12','UBC','FCRL3','PLXNA3','GDNF','VPS11'])

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents
0,GO:BP,GO:0034162,toll-like receptor 9 signaling pathway,0.016195,True,"""The series of molecular signals initiated by ...",19,7,2,21031,0.285714,0.105263,query_1,[GO:0140894]
1,GO:BP,GO:0048486,parasympathetic nervous system development,0.016195,True,"""The process whose specific outcome is the pro...",19,7,2,21031,0.285714,0.105263,query_1,"[GO:0048483, GO:0048731]"
2,GO:MF,GO:1902122,chenodeoxycholic acid binding,0.049903,True,"""Binding to chenodeoxycholic acid."" [GOC:bf, G...",1,7,1,20212,0.142857,1.000000,query_1,"[GO:0005496, GO:0032052]"
3,CORUM,CORUM:5669,PlexinA3-Nrp1 complex,0.049932,True,PlexinA3-Nrp1 complex,2,1,1,3383,1.000000,0.500000,query_1,[CORUM:0000000]
4,CORUM,CORUM:5759,PLXNA3-RANBPM complex,0.049932,True,PLXNA3-RANBPM complex,2,1,1,3383,1.000000,0.500000,query_1,[CORUM:0000000]


In [ ]:
# https://www.nextflow.io/docs/latest/your-first-script.html